[exercises](brir.ipynb)

In [1]:
import soundfile as sf
from scipy.io import loadmat

In [2]:
brir_clap, clap_fs = sf.read("data/brir_clap.wav")
clap_fs

44100

In [3]:
brir_clap.shape

(45998, 2)

In [4]:
len(brir_clap) / clap_fs  # duration in seconds

1.0430385487528344

In [5]:
mat_contents = loadmat("data/brir_sweep.mat", struct_as_record=False, squeeze_me=True)
mat_contents['__header__']

b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Tue May 07 11:52:13 2013'

In [6]:
data = mat_contents['data']
type(data)

scipy.io.matlab.mio5_params.mat_struct

In [7]:
data.SystemLatencySamples, data.SystemlatencyRemoved

(2167, 'YES')

In [8]:
sweep_fs = data.fs
assert sweep_fs == clap_fs
sweep_fs

44100

In [9]:
data.head_azimuth

0

In [10]:
brir_sweep = data.ir
brir_sweep.shape

(66150, 2)

In [11]:
len(brir_sweep) / sweep_fs  # duration in seconds

1.5

In [12]:
speech, speech_fs = sf.read("data/xmas.wav", always_2d=False)
speech.shape

(114400,)

In [13]:
assert speech_fs == clap_fs == sweep_fs
fs = speech_fs

In [14]:
import numpy as np
from scipy import signal
import tools

def convolve_multichannel_ir(x, h, **kwargs):
    """Convolve mono signal x with multichannel impulse reponse h.
    
    This is somewhat inefficient, because the FFT of x is repeated
    multiple times.
    
    After the convolution, the result is normalized to the maximum
    amplitude of x.    
    
    """
    x = np.squeeze(np.asarray(x))
    if x.ndim != 1:
        raise ValueError("x must be a mono signal")
        
    h = np.asarray(h)
    if h.ndim == 1:
        h = h.reshape(-1, 1)
        
    y = np.column_stack([signal.fftconvolve(x, ir, **kwargs) for ir in h.T])
    y = tools.normalize(y, np.max(np.abs(x)))
    return y

Note that `scipy.signal.fftconvolve()` does n-dimensional convolution.
Since we only need one-dimensional convolution, we have to call it with one-dimensional arrays.

Probably, some time in the future, `scipy.signal.fftconvolve()` will grow an `axis` argument ...

https://github.com/scipy/scipy/issues/3525

http://docs.scipy.org/doc/numpy/reference/generated/numpy.apply_along_axis.html

http://stackoverflow.com/a/29678671/500098

http://nipy.org/nitime/api/generated/nitime.utils.html#nitime.utils.fftconvolve

In [15]:
speech_clap = convolve_multichannel_ir(speech, brir_clap)
speech_clap.shape

(160397, 2)

In [16]:
speech_sweep = convolve_multichannel_ir(speech, brir_sweep)

In [17]:
sf.write(speech_clap, "data/xmas_brir_clap.wav", fs)
sf.write(speech_sweep, "data/xmas_brir_sweep.wav", fs)

<audio src="data/xmas.wav" controls></audio>
[data/xmas.wav](data/xmas.wav)

<audio src="data/xmas_brir_clap.wav" controls></audio>
[data/xmas_brir_clap.wav](data/xmas_brir_clap.wav)

<audio src="data/xmas_brir_sweep.wav" controls></audio>
[data/xmas_brir_sweep.wav](data/xmas_brir_sweep.wav)


In [18]:
# more solutions are coming soon!

<p xmlns:dct="http://purl.org/dc/terms/">
  <a rel="license"
     href="http://creativecommons.org/publicdomain/zero/1.0/">
    <img src="http://i.creativecommons.org/p/zero/1.0/88x31.png" style="border-style: none;" alt="CC0" />
  </a>
  <br />
  To the extent possible under law,
  <span rel="dct:publisher" resource="[_:publisher]">the person who associated CC0</span>
  with this work has waived all copyright and related or neighboring
  rights to this work.
</p>